# C. Live Streaming Twitter Alerts

In [ ]:
import numpy as np
import pandas as pd

import tweepy
from tweepy import StreamListener, Stream
import json 
from textblob import TextBlob
from geopy import geocoders 

%matplotlib notebook

from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
#import tmp102
import datetime as dt
from threading import Thread
import time

# input your Twitter API credentials
consumer_key = "CONSUMER KEY"
consumer_secret = "CONSUMER SECRET"
access_token = "ACCESS TOKEN"
access_secret = "ACCESS SECRET"
 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#alert types to pick for filter
alert_fire = [' fire ', ' wildfire ', ' blaze ', ' campfire ', ' smoke ']
alert_flood = [' flood ', ' flooding ', ' flooded ', ' surge ', ' torrent ']
alert_storm = [' storm ', ' tornado ', ' wind ', ' snow ', ' hale ', ' rain ', ' deluge ', ' hurricane ', ' avalanche ', ' ice ']
alert_shooting = [' shooting ', ' shooter ', ' shots fired ', ' bullet ', ' gunshot ', ' murder ', ' driveby ']
alert_any = alert_fire + alert_flood + alert_storm + alert_shooting

# additional filter for urgency - only works on top of other filters
urgent = [' help ', ' HELP ', ' sos ', ' SOS ', ' police ', ' emt ', ' ems ', ' emergency ', 
          ' assistance ', ' assist ', ' 911 ', ' relief ', ' warning ', ' danger ', ' crisis ']

# test set for alerts
shutdown_test = [' shutdown ', ' government ', ' federal ', ' tsa ', 'Trump', ' democrats ',
                'Pelosi', ' republicans ', 'Coulter']

# calculating coordinates of a location
def bounding_coord(city_state, radius=15):
    gn = geocoders.GeoNames(username = 'katerdowdy') 
    for _ in range(3): # setting number of retries
        try:
            coords = gn.geocode(city_state, exactly_one=False)[0]
            lat = coords.latitude
            long = coords.longitude
        except:
            print("Location coordinates not found, try again!")
        
    degree_change = (radius / 69)
    sw_lat = lat - (degree_change / 2)
    sw_long = long - (degree_change / 2)
    ne_lat = lat + (degree_change / 2)
    ne_long = long + (degree_change / 2)
    
    return sw_long, sw_lat, ne_long, ne_lat

# converting unix to Eastern time
def convert_eastern(unix):
    zulu = pd.to_datetime(unix, unit = 'ms')
    eastern = zulu - pd.Timedelta(hours=5)
    round_time = pd.DatetimeIndex.round(eastern, freq = 's')
    return round_time

This part of the streaming API code heavily borrowed from Daniel Foley's [excellent blog post](https://towardsdatascience.com/streaming-twitter-data-into-a-mysql-database-d62a02b050d6) on Towards Data Science. 

In [ ]:
# setting up a class for the StreamListener
class StdOutlistener(StreamListener):
    
    # Setting the alert options
    def __init__(self, alert):
        self.alert = alert
    
    def on_data(self, status):
        all_data = json.loads(status)
        #tweet = TextBlob(all_data["text"])
       
        if any(c in all_data['text'] for c in alert):
            #if any(c in all_data['text'] for c in test):
            #print(("\nFOUND!!\n {}").format(all_data['text']))   
            try:
                print(all_data['text'], all_data['place']['full_name'], 
                      all_data['user']['screen_name'], convert_eastern(all_data['timestamp_ms']))
            except:
                print(all_data['text'], all_data['user']['screen_name'],
                      convert_eastern(all_data['timestamp_ms']))
            with open('./tweetla.json', 'a') as tf:
            # Write a new line
                tf.write('\n\n')
            # Write the json data directly to the file
                json.dump(all_data, tf)
        
            if any(c in all_data['text'] for c in urgent):
                print("CHECK URGENT")
                
            #if any(c in all_data['text'] for c in disaster_preds):
             #   print("DISASTER IMMINENT")
        # Open json text file to save the tweets

        return True

    def on_error(self, status):
        print(status)
        return True

In [ ]:
## INPUTS
city_state = "Topeka, KS" # takes a city, state as one string
radius = 2000 # integer number to set the bounding box
alert = shutdown # type of alert to filter for

# OUTPUTS
coordinates = bounding_coord(city_state=city_state, radius=radius)
# if inputting coordinates manually, input SW long/lat, then NE long/lat
# DC coordinates=[-77.108164, 38.818356, -76.898481, 39.001683])
# continental US coordinates = [-122.673421, 31.232364, -64.434018, 43.791340]

twitterStream = Stream(auth, StdOutlistener(alert))
twitterStream.filter(locations=coordinates)

# Running this cell will show a stream of tweets coming in within the specified area
# along with username, timestamp, and location

We also attempted to build out a flask app to display live Twitter feeds for disasters using code from [Sentdex's repo](https://github.com/Sentdex/socialsentiment/) on a similar project, and with more time to fix some compatibiity issues would probably have been able to get one up and running in thanks to this amazing resource from Sentdex.